In [ ]:
!pip install scikit-fuzzy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 11.3 MB/s eta 0:00:00a 0:00:01


In [5]:
from skfuzzy import control as ctrl
import numpy as np
import skfuzzy as fuzz

condition_severity = ctrl.Antecedent(np.arange(0, 101, 1), 'condition_severity')
symptom_relevance = ctrl.Antecedent(np.arange(0, 101, 1), 'symptom_relevance')
recommendation = ctrl.Consequent(np.arange(0, 101, 1), 'recommendation')

condition_severity['low'] = fuzz.trapmf(condition_severity.universe, [0, 0, 30, 50])
condition_severity['medium'] = fuzz.trimf(condition_severity.universe, [30, 50, 70])
condition_severity['high'] = fuzz.trapmf(condition_severity.universe, [50, 70, 100, 100])

symptom_relevance['low'] = fuzz.trapmf(symptom_relevance.universe, [0, 0, 30, 50])
symptom_relevance['medium'] = fuzz.trimf(symptom_relevance.universe, [30, 50, 70])
symptom_relevance['high'] = fuzz.trapmf(symptom_relevance.universe, [50, 70, 100, 100])

recommendation['routine'] = fuzz.trapmf(recommendation.universe, [0, 0, 30, 50])
recommendation['priority'] = fuzz.trimf(recommendation.universe, [30, 50, 70])
recommendation['urgent'] = fuzz.trapmf(recommendation.universe, [50, 70, 100, 100])

rule1 = ctrl.Rule(condition_severity['high'] & symptom_relevance['high'], recommendation['urgent'])
rule2 = ctrl.Rule(condition_severity['medium'] & symptom_relevance['medium'], recommendation['priority'])
rule3 = ctrl.Rule(condition_severity['low'] & symptom_relevance['low'], recommendation['routine'])

recommendation_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])




In [9]:
recommendation_sim = ctrl.ControlSystemSimulation(recommendation_ctrl)
recommendation_sim.input['condition_severity'] = 80
recommendation_sim.input['symptom_relevance'] = 70
recommendation_sim.compute()

print(recommendation_sim.output['recommendation'])

79.58333333333333


In [13]:
import os

# Define the path where .hea files are stored
hea_files_directory = '../pred_res/input_data/train'
attribute_counts = {"Dx": 0, "Rx": 0, "Hx": 0, "Sx": 0}

# Loop through all .hea files in the directory
for filename in os.listdir(hea_files_directory):
    if filename.endswith('.hea'):
        # Open and read the file
        with open(os.path.join(hea_files_directory, filename), 'r') as file:
            content = file.read()
            
            # Check for the presence of each attribute
            if "#Dx: Unknown" not in content:
                attribute_counts["Dx"] += 1
            if "#Rx: Unknown" not in content:
                attribute_counts["Rx"] += 1
            if "#Hx: Unknown" not in content:
                attribute_counts["Hx"] += 1
            if "#Sx: Unknown" not in content:
                attribute_counts["Sx"] += 1

# Print the count of files with each attribute
print("Presence of attributes across .hea files:")
for attribute, count in attribute_counts.items():
    print(f"{attribute}: {count} files")

# Optional: Print total files processed for context
total_files = len([f for f in os.listdir(hea_files_directory) if f.endswith('.hea')])
print(f"Total .hea files processed: {total_files}")

Presence of attributes across .hea files:
Dx: 38788 files
Rx: 0 files
Hx: 0 files
Sx: 0 files
Total .hea files processed: 38788


In [14]:
!nvidia-smi

Sun Oct 27 01:46:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10                     On  | 00000000:81:00.0 Off |                    0 |
|  0%   36C    P0              62W / 150W |  17624MiB / 23028MiB |      8%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
import pandas as pd
# Get top record from predictions.csv
top_record = pd.read_csv('predictions.csv', header=0, nrows=3)
print("Top record from predictions.csv:")
print(top_record)

Top record from predictions.csv:
      filename  10370003_label  ...  713426002_score  713427006_score
0   E03065.mat               1  ...              0.0              1.0
1   E03604.mat               1  ...              0.0              1.0
2  HR15478.mat               1  ...              0.0              1.0

[3 rows x 49 columns]


In [30]:
from collections import defaultdict
record_dict = defaultdict(dict)

In [31]:
for key, value in top_record.iloc[0].items():
    if key != 'filename':
        record_key = key.split('_')
        record_dict[record_key[0]][record_key[1]] = value
    else:
        record_dict[key] = value

In [35]:
import json
record_dict

defaultdict(dict,
            {'filename': 'E03065.mat',
             '10370003': {'label': 1, 'score': 1.0},
             '111975006': {'label': 0, 'score': 0.0},
             '164889003': {'label': 1, 'score': 1.0},
             '164890007': {'label': 1, 'score': 1.0},
             '164909002': {'label': 0, 'score': 0.0},
             '164917005': {'label': 0, 'score': 0.0},
             '164934002': {'label': 1, 'score': 1.0},
             '164947007': {'label': 0, 'score': 0.0},
             '251146004': {'label': 1, 'score': 1.0},
             '270492004': {'label': 0, 'score': 2.5726672e-36},
             '284470004': {'label': 1, 'score': 1.0},
             '39732003': {'label': 0, 'score': 1.9047508e-24},
             '426177001': {'label': 1, 'score': 1.0},
             '426627000': {'label': 0, 'score': 0.0},
             '426783006': {'label': 1, 'score': 1.0},
             '427084000': {'label': 1, 'score': 1.0},
             '427172004': {'label': 0, 'score': 0.0},
       

In [ ]:
for key, value in record_dict.items():
    try:
        label = value['label']
        score = value['score']
        confidence =0.6
        record_dict[key]["strength"] = score * confidence if label else (1 - score) * confidence
    except KeyError:
        print(f"{key}: {value}")

In [36]:
from owlready2 import *
from owlready2.pymedtermino2 import *
from owlready2.pymedtermino2.umls import *

default_world.set_backend(filename="pym.sqlite3")

In [ ]:
import_umls("umls-2024AA-metathesaurus-full.zip", terminologies=["SNOMEDCT_US"])
default_world.save()